# Mallorn: Edge Model Delta Updates

This notebook demonstrates the complete workflow for creating and applying model patches using Mallorn.

**Key Benefits:**
- 95%+ bandwidth reduction for OTA updates
- ~10ms fingerprinting for version detection
- Ed25519 signing for security

## Setup

Install mallorn if not already installed:

In [ ]:
# pip install mallorn
import mallorn
print(f"Mallorn version: {mallorn.__doc__}")

## 1. Create Sample Models

For demonstration, we'll create two synthetic "models" (in practice, these would be TFLite, GGUF, or ONNX files).

In [ ]:
import numpy as np
import tempfile
import os

# Create temporary directory for our files
tmpdir = tempfile.mkdtemp(prefix="mallorn_demo_")

# Simulate a model file (16 MB of weights)
np.random.seed(42)
model_v1_weights = np.random.randn(4 * 1024 * 1024).astype(np.float32)

# V2 is a "fine-tuned" version (small changes)
model_v2_weights = model_v1_weights.copy()
# Simulate fine-tuning: small adjustments to 10% of weights
mask = np.random.random(len(model_v2_weights)) < 0.1
model_v2_weights[mask] += np.random.randn(mask.sum()).astype(np.float32) * 0.01

# Save as raw binary (in practice, use TFLite/GGUF/ONNX)
v1_path = os.path.join(tmpdir, "model_v1.bin")
v2_path = os.path.join(tmpdir, "model_v2.bin")
patch_path = os.path.join(tmpdir, "update.patch")

model_v1_weights.tofile(v1_path)
model_v2_weights.tofile(v2_path)

print(f"Model V1: {os.path.getsize(v1_path):,} bytes")
print(f"Model V2: {os.path.getsize(v2_path):,} bytes")

## 2. Model Fingerprinting

Before updating, we need to identify which version a device has. Fingerprinting is fast (~10ms) regardless of model size.

In [ ]:
import time

# Fingerprint both models
start = time.time()
fp_v1 = mallorn.fingerprint(v1_path)
elapsed = time.time() - start

print(f"V1 Fingerprint:")
print(f"  Size: {fp_v1.file_size:,} bytes")
print(f"  Short ID: {fp_v1.short()}")
print(f"  Time: {elapsed*1000:.2f}ms")

fp_v2 = mallorn.fingerprint(v2_path)
print(f"\nV2 Fingerprint:")
print(f"  Size: {fp_v2.file_size:,} bytes")
print(f"  Short ID: {fp_v2.short()}")

In [ ]:
# Compare fingerprints
if mallorn.compare_fingerprints(v1_path, v2_path):
    print("Models are the same version")
else:
    print("Models are different - update needed!")

## 3. Create a Patch

Create a delta patch from V1 to V2. This captures only the differences.

In [ ]:
# Create patch with neural-aware compression (best for fine-tuned models)
stats = mallorn.create_patch(
    v1_path,
    v2_path,
    patch_path,
    compression_level=9,
    neural=True
)

print(f"Patch Statistics:")
print(f"  Source size: {stats.source_size:,} bytes")
print(f"  Target size: {stats.target_size:,} bytes")
print(f"  Patch size:  {stats.patch_size:,} bytes")
print(f"  Compression: {stats.compression_ratio:.1f}x")
print(f"  Bandwidth savings: {(1 - stats.patch_size / stats.target_size) * 100:.1f}%")

## 4. View Patch Info

In [ ]:
info = mallorn.patch_info(patch_path)

print(f"Patch Details:")
print(f"  Format: {info.format}")
print(f"  Version: {info.version}")
print(f"  Source hash: {info.source_hash[:16]}...")
print(f"  Target hash: {info.target_hash[:16]}...")
print(f"  Compression: {info.compression}")
print(f"  Operations: {info.operation_count}")

## 5. Apply the Patch

Simulate what happens on the device: apply the patch to V1 to get V2.

In [ ]:
restored_path = os.path.join(tmpdir, "model_v2_restored.bin")

result = mallorn.apply_patch(v1_path, patch_path, restored_path)

print(f"Patch Application Result:")
print(f"  Source valid: {result.source_valid}")
print(f"  Patch valid: {result.patch_valid}")
print(f"  Overall: {'SUCCESS' if result.is_valid() else 'FAILED'}")

In [ ]:
# Verify the restored model matches the original V2
import hashlib

def file_hash(path):
    with open(path, 'rb') as f:
        return hashlib.sha256(f.read()).hexdigest()[:16]

original_hash = file_hash(v2_path)
restored_hash = file_hash(restored_path)

print(f"Original V2 hash:  {original_hash}")
print(f"Restored V2 hash:  {restored_hash}")
print(f"Match: {'YES' if original_hash == restored_hash else 'NO'}")

## 6. Verify Before Applying

On resource-constrained devices, verify the patch can be applied before committing.

In [ ]:
# Verification without applying
verify_result = mallorn.verify_patch(v1_path, patch_path)

print(f"Verification:")
print(f"  Source matches: {verify_result.source_valid}")
print(f"  Patch integrity: {verify_result.patch_valid}")
print(f"  Expected output hash: {verify_result.expected_target_hash[:16]}...")

## 7. Cleanup

In [ ]:
import shutil
shutil.rmtree(tmpdir)
print(f"Cleaned up {tmpdir}")

## Summary

| Operation | Function | Typical Time |
|-----------|----------|-------------|
| Fingerprint | `mallorn.fingerprint()` | ~10ms |
| Compare | `mallorn.compare_fingerprints()` | ~20ms |
| Create Patch | `mallorn.create_patch()` | ~1-10s |
| Apply Patch | `mallorn.apply_patch()` | ~0.5-5s |
| Verify | `mallorn.verify_patch()` | ~100ms |

Typical bandwidth savings: **90-99%** for fine-tuned models.